# Variance Reduction: Control Variates and Antithetic Variables

In [1]:
import math
import numpy as np
import scipy.stats as stats 
import seaborn as sns
import matplotlib.pyplot as plt

## Simulation: Inventory

In [2]:
## parameters for inventory system
lmbda = 100 # mean demand size
c = 2 # sales price
h = 0.1 # inventory cost per item
K = 10 # fixed ordering cost
k = 1 # marginal ordering cost per item
p = 0.95 # probability that order arrives
m = 30 # number of days

# simple generator for Poisson distribution
# using inverse transform method
def GeneratePoisson(lmbda):
    X = 0
    sum = math.exp(-lmbda)
    prod = math.exp(-lmbda)
    U = np.random.uniform()
    while U > sum:
        X = X + 1
        prod = prod * lmbda / X
        sum = sum + prod
    return X
    
def SimulateOneRun(s,S):
    X = S
    profit = 0
    total_demand = 0
    # compute a realization of average profit
    # return both the profit
    # and the control variate (demand)
    for j in range(m):
        demand = GeneratePoisson(lmbda)
        total_demand = total_demand + demand
        sales = min(X, demand)
        Y = X - sales
        U = np.random.uniform()
        if (Y < s) and (U < p):
            profit = profit - (K + k * (S-Y))
            X = S
        else:
            X = Y
        profit = profit + c*sales - h*X
    ave_profit = profit/m
    ave_demand = total_demand/m
    return ave_profit, ave_demand

## Run simulation and get estimates (with/without control variate)

In [36]:
np.random.seed(2025)

# compute average profits using (80,200) policy
n_reps = 50
s = 80
S = 200
results = [SimulateOneRun(s,S) for i in range(n_reps)]
X = np.array([r[0] for r in results])
Y = np.array([r[1] for r in results])

# control variate based estimator
C_xy = np.cov(X,y=Y,ddof=1)[0,1]
var_y = np.var(Y,ddof=1)
a = C_xy/var_y
cv_est = X - a*(Y - lmbda) # true mean of Y - poisson mean

In [37]:
# usual estimator
print("The estimate from usual procedure is {}".format(np.mean(X)))
# control variate based estimator
print("The control variate based estimate is {}".format(np.mean(cv_est)))

The estimate from usual procedure is 76.75786666666667
The control variate based estimate is 76.78947130462213


In [38]:
# variance of the usual estimate
print("Variance of usual estimator is {}".format(np.var(X)))
# variance of the control variate estimate
print("Variance of control variate based estimator is {}".format(np.var(cv_est)))

Variance of usual estimator is 11.836950560000005
Variance of control variate based estimator is 11.108595724204456


In [39]:
# confidence interval from the usual estimate
ci_usual = stats.ttest_1samp(X, popmean=0).confidence_interval(confidence_level=0.95)
print("CI using usual estimator: ({},{}), width {}".format(ci_usual.low,ci_usual.high,ci_usual.high-ci_usual.low))
# confidence interval from the control variate estimate
ci_cv = stats.ttest_1samp(cv_est, popmean=0).confidence_interval(confidence_level=0.95)
print("CI using control variate based estimator: ({},{}), width {}".format(ci_cv.low,ci_cv.high,ci_cv.high-ci_cv.low))

CI using usual estimator: (75.77016417958092,77.74556915375243), width 1.9754049741715107
CI using control variate based estimator: (75.83263904005634,77.74630356918792), width 1.9136645291315801


## Run simulation and get estimates (using antithetic variables)

In [43]:
# simple generator of antithetic variables
# for Poisson distribution
# using inverse transform method
def GeneratePoissonAV(lmbda):
    # First Poisson sample using U1
    X1 = 0
    sum1 = math.exp(-lmbda)
    prod1 = math.exp(-lmbda)
    U1 = np.random.uniform()
    while U1 > sum1:
        X1 += 1
        prod1 = prod1 * lmbda / X1
        sum1 += prod1

    # Antithetic Poisson sample using 1 - U1
    X2 = 0
    sum2 = math.exp(-lmbda)
    prod2 = math.exp(-lmbda)
    U2 = 1 - U1
    while U2 > sum2:
        X2 += 1
        prod2 = prod2 * lmbda / X2
        sum2 += prod2

    return X1, X2


# simulate using antithetic variables
def SimulateOneAVRun(s,S):
    X1 = S
    X2 = S
    profit1 = 0
    profit2 = 0
    # compute a pair of profit realizations using antithetic variables
    for j in range(m):
        demand1, demand2 = GeneratePoissonAV(lmbda)
        sales1 = min(X1, demand1)
        sales2 = min(X1, demand2)
        Y1 = X1 - sales1
        Y2 = X2 - sales2
        U1 = np.random.uniform()
        U2 = 1 - U1
        if (Y1 < s) and (U1 < p):
            profit1 = profit1 - (K + k * (S-Y1))
            X1 = S
        else:
            X1 = Y1
        profit1 = profit1 + c*sales1 - h*X1
        if (Y2 < s) and (U2 < p):
            profit2 = profit2 - (K + k * (S-Y2))
            X2 = S
        else:
            X2 = Y2
        profit2 = profit2 + c*sales2 - h*X2
    ave_profit = (profit1+profit2)/(2*m)
    return ave_profit

In [44]:
np.random.seed(2025)
# compute average profits using (80,200) policy
# (from before, with and without control variates)
n_reps = 50
s = 80
S = 200
results = [SimulateOneRun(s,S) for i in range(n_reps)]
X = np.array([r[0] for r in results])
Y = np.array([r[1] for r in results])

# control variate based estimator
C_xy = np.cov(X,y=Y)[0,1]
var_y = np.var(Y)
a = C_xy/var_y
cv_est = X - a*(Y-lmbda)

In [45]:
# compute average profits using (80,200) policy
# (using antithetic variables)
n_reps = 50
s = 80
S = 200
results = np.array([SimulateOneAVRun(s,S) for i in range(n_reps)])

In [47]:
# usual estimator
print("The estimate from usual procedure is {}".format(np.mean(X)))
# control variate based estimator
print("The control variate based estimate is {}".format(np.mean(cv_est)))
# antithetic variable variate based estimator
print("The antithetic variable based estimate is {}".format(np.mean(results)))

The estimate from usual procedure is 76.75786666666667
The control variate based estimate is 76.79011629723347
The antithetic variable based estimate is 77.18316666666668


In [48]:
# variance of the usual estimate
print("Variance of usual estimator is {}".format(np.var(X)))
# variance of the control variate estimate
print("Variance of control variate based estimator is {}".format(np.var(cv_est)))
# variance of the antithetic variable estimate
print("Variance of antithetic variable based estimator is {}".format(np.var(results)))

Variance of usual estimator is 11.836950560000005
Variance of control variate based estimator is 11.108899078988197
Variance of antithetic variable based estimator is 6.424247694444443


In [36]:
# confidence interval from the usual estimate
ci_usual = stats.ttest_1samp(X, popmean=0).confidence_interval(confidence_level=0.95)
print("CI using usual estimator: ({},{}), width {}".format(ci_usual.low,ci_usual.high,ci_usual.high-ci_usual.low))
# confidence interval from the control variate estimate
ci_cv = stats.ttest_1samp(cv_est, popmean=0).confidence_interval(confidence_level=0.95)
print("CI using control variate based estimator: ({},{}), width {}".format(ci_cv.low,ci_cv.high,ci_cv.high-ci_cv.low))
# confidence interval from the antithetic variable estimate
ci_av = stats.ttest_1samp(results, popmean=0).confidence_interval(confidence_level=0.95)
print("CI using antithetic variable based estimator: ({},{}), width {}".format(ci_av.low,ci_av.high,ci_av.high-ci_av.low))

CI using usual estimator: (75.77016418087848,77.74556915245486), width 1.9754049715763813
CI using control variate based estimator: (75.83327096937226,77.74696162509467), width 1.9136906557224052
CI using antithetic variable based estimator: (76.45552532058242,77.91080801275093), width 1.4552826921685096
